In [ ]:
%pip install ultralytics

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, GlobalAveragePooling2D
!cd ..
import keras.backend as k
from keras.layers import Lambda

from ultralytics import YOLO



import matplotlib.pyplot as plt
import os
import numpy as np

classif = tf.keras.models.load_model('saved_model/Feature_Extractor_Model')
model = YOLO('./Yolov8_minutiae_detection.pt')
model_vgg16 = VGG16(weights="imagenet", include_top="false")
FeatureExtractor_model = Model(inputs=model_vgg16.inputs, outputs=model_vgg16.layers[-5].output)

In [2]:
def extract_features_from_single_image(image_path):
    # Load the image
    image = load_img(image_path, target_size=(224, 224))
    image_array = img_to_array(image)
    image_array = preprocess_input(image_array)  # Apply appropriate preprocessing
    
    # Reshape and extract features
    image_features = FeatureExtractor_model.predict(np.expand_dims(image_array, axis=0))
    
    return image_features

In [5]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
def detect_minuities(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = fingerprint_enhancer.enhance_Fingerprint(img)
    image_height, image_width,_ = img.shape
    imag = 255 *np.ones((image_height, image_width, 3), dtype=np.uint8)
    results = model.predict(img)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            b = box.xyxy[0].cpu().numpy() # get box coordinates in (top, left, bottom, right) format
            c = box.cls.cpu().numpy()[0]
            top = b[0]
            left = b[1]
            bottom = b[2]
            right = b[3]
            cv2.rectangle(imag, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 2)
            imag = cv2.putText(imag, Labels[int(c)], (int(left), int(top)),
                        cv2.FONT_HERSHEY_PLAIN, 1, (220,20,60), 1)
    return imag

In [11]:
from flask import Flask, request, jsonify
import base64

app = Flask(__name__)

@app.route('/PassImage', methods=['POST'])
def receive_image():
    try:
        # Get the JSON data from the request
        json_data = request.get_json()
        base64_image = json_data.get('image', '')
        
        # Decode the base64 string into bytes
        image_data = base64.b64decode(base64_image)

        # Save the image data to a file
        with open("imageToSave.png", "wb") as fh:
            fh.write(image_data)
            
        # Extract features from the single image
        single_image_features = extract_features_from_single_image("imageToSave.png")

        # Reshape the features to match the expected input shape of your prediction model
        reshaped_features = np.reshape(single_image_features, (1, 7 * 7 * 512))

        # Use the trained model to predict using the extracted features
        prediction = classif.predict(reshaped_features)

        # Convert the prediction probability to a binary label
        predicted_label = "Real" if prediction > 0.5 else "Fake"

        # Print the prediction
        print("Predicted pourcentage:", prediction)
        print("Predicted Label:", predicted_label)
        res = 'success'
        if predicted_label == "Real":
            res = 'success'      
            img = detect_minuities("imageToSave.png")
            image_filename = 'processed_image.png'
            cv2.imwrite(image_filename,img)
        else:
            res = 'err'
        return jsonify({'message': 'predicted_label', 'status': res, 'image_filename': image_filename}), 200
    except Exception as e:
        print(e)
        return jsonify({'error': str(e), 'status': 'error'}), 400

@app.route('/get_image/<image_filename>', methods=['GET'])
def get_image(image_filename):
    try:
        # Retrieve and send the processed image to Android
        return send_file(image_filename, mimetype='image/png')
    except Exception as e:
        return jsonify({'error': str(e), 'status': 'error'})
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.17:5000
Press CTRL+C to quit


1/1 [==============================] - 0s 26ms/step


192.168.1.17 - - [27/Sep/2023 14:08:58] "POST /PassImage HTTP/1.1" 200 -


Predicted pourcentage: [[  2.267e-36]]
Predicted Label: Real
1/1 [==============================] - 0s 30ms/step


...

Predicted pourcentage: [[          0]]
Predicted Label: Fake
1/1 [==============================] - 0s 32ms/step


192.168.1.17 - - [27/Sep/2023 14:31:03] "POST /PassImage HTTP/1.1" 200 -


Predicted pourcentage: [[          1]]
Predicted Label: Real
1/1 [==============================] - 0s 27ms/step


192.168.1.17 - - [27/Sep/2023 14:31:16] "POST /PassImage HTTP/1.1" 200 -


Predicted pourcentage: [[          0]]
Predicted Label: Fake


In [13]:
from ultralytics.yolo.utils.plotting import Annotator
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np

model = YOLO('./best.pt')
img = cv2.imread('./101_2_jpg.rf.27d1ff8fd950cee6f3b4b1b4e3d8a598.jpg')
img = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
image_height, image_width,_ = img.shape
image = np.zeros((image_height, image_width, 3), dtype=np.uint8)
results = model.predict(img)


Ultralytics YOLOv8.0.20  Python-3.9.12 torch-1.13.1 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs


In [14]:
print(results)

[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([0, 6])
dtype: torch.float32
 + tensor([], device='cuda:0', size=(0, 6))]


In [15]:
for r in results:
    print(r)
            

tensor([], device='cuda:0', size=(0, 6))


In [16]:
import fingerprint_enhancer                                # Load the library

img = cv2.imread('./Fingerprint_DS/train/Real/1__M_Left_little_finger.BMP', 0)                      # read input image

out = fingerprint_enhancer.enhance_Fingerprint(img)        # enhance the fingerprint image

cv2.imshow('enhanced_image', out);                     # display the result

cv2.waitKey(0)

-1